In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#impoer selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

In [8]:
base_url = 'https://www.booking.com'

In [9]:
df = pd.read_csv('../data/raw/city_attractions.csv')
df.head()

,city,name,description,duration,rating,reviews,price,link,image_url
0,New York,SUMMIT One Vanderbilt Tickets,Digital art installations and views of New Yor...,Duration: 2 hours,4.7,4.7,CAD 63.57,/attractions/us/prcmokyuz7um-summit-one-vander...,https://r-xx.bstatic.com/xdata/images/xphoto/3...
1,New York,9/11 Memorial & Museum Admission,Chance to visit a memorial and museum that's d...,NaN,4.8,4.8,CAD 36.66,/attractions/us/prgstbyhf5aj-911-memorial-muse...,https://r-xx.bstatic.com/xdata/images/xphoto/3...
2,New York,New York CityPASS,A sightseeing pass to explore Big Apple attrac...,NaN,4.5,4.5,CAD 198.23,/attractions/us/prj7dhkaezyz-new-york-city-pas...,https://r-xx.bstatic.com/xdata/images/xphoto/3...
3,New York,One-hour Sightseeing Yacht Cruise,A narrated cruise with views of the Statue of ...,NaN,3.8,3.8,CAD 21.72,/attractions/us/prulfebtv6ii-one-hour-sightsee...,https://q-xx.bstatic.com/xdata/images/xphoto/3...
4,New York,SUMMIT One Vanderbilt Ticket,Opportunity to go on an immersive multisensory...,NaN,4.5,4.5,CAD 63.57,/attractions/us/pr3zhwuklbbp-summit-one-vander...,https://q-xx.bstatic.com/xdata/images/xphoto/3...


In [10]:
driver = webdriver.Chrome()

In [11]:
def scrape_content(city_name, url):

    driver.get(url)
    wait = WebDriverWait(driver, 20)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    desc = soup.find('div', class_='css-n9kgwt')
    desc = desc.text if desc is not None else ""
    
    why_visit_div = soup.find('div', class_='css-1a8s82q')
    why_visit_text = ''
    if why_visit_div is not None:
        why_visit = why_visit_div.find_all('li', class_='f660aace8b f846ce2e52')
        if why_visit is not None:
            why_visit_text = [li.text for li in why_visit]
    
    
    whats_included_div = soup.find('div', class_='bcdcb105b3 f660aace8b css-51s7b5')
    whats_included_text = ''
    if whats_included_div is not None:
        whats_included = whats_included_div.find_all('li', class_='f660aace8b f846ce2e52')
        if whats_included is not None:
            whats_included_text = [li.text for li in whats_included]
            
    whats_not_included_div = soup.find_all('div', class_='bcdcb105b3 f660aace8b css-51s7b5')
    whats_not_included_text = ''
    if whats_not_included_div is not None and len(whats_not_included_div) > 1:
        whats_not_included = whats_not_included_div[1].find_all('li', class_='f660aace8b f846ce2e52')
        if whats_not_included is not None:
            whats_not_included_text = [li.text for li in whats_not_included]
            
            
    restrictions_div = soup.find_all('div', class_='css-1a8s82q')
    restrictions_text = ''
    if restrictions_div is not None and len(restrictions_div) > 1:
        restrictions = restrictions_div[1].find_all('li', class_='a8b57ad3ff d50c412d31 fb9a5438f9 c7a5a1307a')
        if restrictions is not None:
            restrictions_text = [li.text for li in restrictions]
            print(restrictions_text)
            
    
    additional_div = soup.find_all('div', class_='css-1a8s82q')
    additional_text = ''
    if additional_div is not None and len(additional_div) > 2:
        additional = additional_div[2].find_all('div')
        if additional is not None:
            additional_text = [li.text for li in additional]
            print(additional_text)
            
    
    location_div = soup.find('div', class_='f660aace8b f81ab4937d')
    location_text = location_div.text if location_div is not None else ""
    
    return {
        'city_name': city_name,
        'description': desc,
        'why_visit': why_visit_text,
        'whats_included': whats_included_text,
        'whats_not_included': whats_not_included_text,
        'restrictions': restrictions_text,
        'additional_info': additional_text,
        'location': location_text
    }    
    

In [12]:
all_attractions = []
for i, destination in df.iterrows():
    city_name = destination['name']
    attraction_url = base_url + destination['link']
    city_attraction = scrape_content(city_name, attraction_url)
    all_attractions.append(city_attraction)

['Comfortable footwear is recommended.']
['Children aged 0–5 years can join for free.\rPlease note that some rooms and installations have mirrored floors. Please dress comfortably and in accordance with the tour. Trousers, shorts or tights are recommended. Sunglasses are recommended for day visits. Note that stiletto heels, steel-toe boots and any footwear that could cause damage to the mirrored floors are not allowed on this tour.\rPlease note that there is no storage for personal items. Large bags, large backpacks and luggage are not permitted on this tour.\rProfessional photographs are available to purchase in print and digital format at the end of the experience.\rPlease bring your ticket with you to the attraction.\rShow more', 'Show more']


IndexError: list index out of range

In [ ]:
all_attractions

[{'city_name': 'SUMMIT One Vanderbilt Tickets',
  'description': 'With this ticket to the Summit One Vanderbilt, you’ll see futuristic digital shows inside one of New York’s newest skyscrapers. You’ll also take in views of Manhattan from a height of 370 metres. \r As you explore three levels of multi-sensory galleries inside the skyscraper, you’ll see digital art installations by internationally acclaimed artists and designers. You’ll see works inspired by stories, optical illusions and interactivity. You’ll also see the busy streets below the building during the ‘Levitation’ experience, which involves standing on a transparent platform over 300 metres above the ground. \r When you reach the SUMMIT Terrace, you can see New York’s skyline and views for up to 80 miles. You can also order cocktails and light fare from Café Après.',
  'why_visit': ['Three floors of multi-sensory immersive experiences at the top of a skyscraper',
   'Panoramic views of New York and landmarks like the Empire

In [ ]:
df = pd.DataFrame(all_attractions)
print(df.shape)
df.head()

(1, 8)


,city_name,description,why_visit,whats_included,whats_not_included,restrictions,additional_info,location
0,SUMMIT One Vanderbilt Tickets,"With this ticket to the Summit One Vanderbilt,...",[Three floors of multi-sensory immersive exper...,"[Admission, Levitation experience]","[Ascent experience, Food and drinks]",[Comfortable footwear is recommended.],[Children aged 0–5 years can join for free.\rP...,SUMMIT One Vanderbilt Tickets45 East 42nd Stre...


In [ ]:
df.to_csv('../data/raw/city_attractions_details.csv', index=False)